In [1]:
import pandas as pd
import numpy as np

# Read Excel file
file_path = 'NPS_onboarding.xlsx'
sheet_name = 'pre-processed'
df = pd.read_excel(file_path, sheet_name=sheet_name)

# Define categories
categories = ['Kejelasan Informasi', 'Bimbingan dan Arahan', 'Responsif dalam Menjawab Pertanyaan', 'Efektifitas Waktu','Kepuasan Student']

# Create a dictionary to store results
nps_results = {}

# Calculate NPS for each category
for category in categories:
    # Classify responses
    df['Category'] = np.where(df[category] < 5, 'Detractor', np.where(df[category] >= 5, 'Promoter', 'Passive'))

    # Calculate NPS
    nps_df = df.groupby(['Batch', 'Category']).size().unstack(fill_value=0)
    nps_df['Total'] = nps_df.sum(axis=1)
    nps_df[f'NPS_{category}'] = (nps_df['Promoter'] / nps_df['Total']) - (nps_df['Detractor'] / nps_df['Total'])

    # Store results
    nps_results[category] = nps_df

# Create a new DataFrame for the results
nps_summary_df = pd.DataFrame()

# Update the new sheet with NPS results for each category
for category, result_df in nps_results.items():
    result_df.reset_index(inplace=True)
    result_df = result_df.rename(columns={f'NPS_{category}': f'{category}_NPS'})  # Rename NPS column for clarity
    if nps_summary_df.empty:
        nps_summary_df = result_df
    else:
        nps_summary_df = pd.merge(nps_summary_df, result_df[['Batch', f'{category}_NPS']], on='Batch', how='outer')

# Calculate the average NPS score
nps_columns = [f'{category}_NPS' for category in categories]
nps_summary_df['Average_NPS'] = nps_summary_df[nps_columns].mean(axis=1)

# Display the results
print(nps_summary_df)


Category      Batch  Detractor  Promoter  Total  Kejelasan Informasi_NPS  \
0        2024-02-01         42       568    610                 0.862295   

Category  Bimbingan dan Arahan_NPS  Responsif dalam Menjawab Pertanyaan_NPS  \
0                          0.84918                                 0.822951   

Category  Efektifitas Waktu_NPS  Kepuasan Student_NPS  Average_NPS  
0                      0.760656              0.796721     0.818361  
